# Notebook 10: Frequency-Domain Specifications and Loop Shaping
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores frequency-domain specifications and loop shaping approaches for control system design, covering concepts like gain and phase margins, performance limitations, and systematic control synthesis methods.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch


# Learning Objectives


After completing this material, you should be able to:

1. Understand and specify control requirements in the frequency domain:
   - Express command tracking requirements 
   - Specify disturbance rejection requirements
   - Define noise rejection specifications

2. Use loop shaping techniques for control synthesis:
   - Apply proportional control effectively
   - Design lead compensation for phase margin improvement
   - Design lag compensation for steady-state error reduction
   - Combine compensation elements systematically

3. Understand fundamental performance limitations:
   - Recognize the waterbed effect
   - Handle constraints from non-minimum phase zeros
   - Deal with limitations from unstable poles



## Required Packages



Run the following cell to import required packages:

In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider, HBox, VBox, Checkbox
from IPython.display import display, clear_output, HTML
from scipy import signal
import control

# 1. Frequency-Domain Specifications


In the past few weeks we have introduced the frequency response, and corresponding tools (Bode, Nyquist) that we can use to assess both closed-loop stability, stability margins and system behavior. However, we had not yet seen how we could use the frequency responses to determine whether the desired behavior is achieved. Thus, this week we introduce frequency-domain specifications, the frequency equivalent of the time-domain specifications seen previously, along with how they can be used to achieve the desired system behavior.

## 1.1 Introduction

So far we have considered (time) specifications in terms of output responses -- e.g., ensure that the system achieves $M\%$ overshoot with a rise time of $X$ seconds with no steady state error. However, control systems are typically exposed to disturbances and noise, which may correspondingly affect the ability to achieve such specifications. As an intuitive example, consider that we wish to regulate the temperature of a room. If the noise of the temperature sensor is large, it could result in the thermostat continuously adjusting the temperature of the room, and thus the room temperature may never reach its desired temperature.

Disturbance and noise signals can be typically expressed in terms of some frequency band -- e.g., think that the noise from a sensor typically occurs at a high-frequency (greater than $100Hz$), whereas disturbances, such as the draft entering from below a door, typically occur at low frequencies (less than $10Hz$). Consequentially, we can express disturbance rejection and noise rejection by through distinct frequency bands:

- *Low frequencies*: At low frequencies we investigate command tracking and disturbance rejection.
- *High frequencies*: At high frequencies we investigate noise rejection 
- *Mid-frequencies*: In the middle of the frequency range we need to consider the stability margins and robustness


Before introducing the frequency specifications, recall that for a closed-loop system with the below feedback architecture, we work with two key transfer functions:
<div style="text-align:center;">
<img src="./img/feedback_architecture.png" alt="Obstacle" width="500">
</div>

1. Sensitivity function: $S(s) = \frac{1}{1 + L(s)}$
   - $S(s)$ captures the system's ability to reject disturbances and determines error in tracking commands
2. Complementary sensitivity function: $T(s) = \frac{L(s)}{1 + L(s)}$
   - $T(s)$ captures the system's ability to track commands and determines the effect of noise.

Where $L(s) = P(s)C(s)$ is the loop transfer function.


## 1.2 Frequency Specifications


Frequency specifications are typically phrased in terms of achieving, good command tracking/disturbance rejection and noise rejection, and can be achieved by:

1. Command tracking/disturbance rejection:
   - $|S(j\omega)| \ll 1$ at low frequencies, 
   - Thus $|L(j\omega)|$ is large at low frequencies and hence $|T(j\omega)|\approx 1$ at low frequencies.
   - Example: *ensure commands are tracked with max 10% error up to 10Hz*

2. Noise rejection:
   - $|T(j\omega)| \ll 1$ at high frequencies  
   - Example: *ensure noise is reduced by factor of 10 at frequencies above 100Hz*

The above specifications can be quite generic in general and mixed and matched with time-domain specification as well as the phase and gain margins -- e.g., have a rise time of $X$ seconds whilst maintaining a phase margin of $Y^\circ$. Although for certain specifications it may be possible to derive expressions for specific systems (i.e., with what we did for time-domain specifications of $1^{\text{st}}$ and $2^{\text{nd}}$ order systems), approximations are typically performed as a rule of thumb as they tend to hold, and the results can then be easily verified. Consequentially, if non-standard approximations are required, these will be provided in the question -- see the problems in the corresponding problem sheet for examples on this.

The above specifications are written in terms of the closed-loop transfer function. However, we have introduced tools that plot the open-loop transfer function, from which we can then infer the closed-loop behavior. Thus, in the following section we make this connection.


## 1.3 Translating to Open-Loop Specifications (Bode Obstacle Course)

The closed-loop specifications can be translated to requirements on the open-loop transfer function $L(s)$:

1. Command tracking/disturbance rejection requirement:
   - Recall that at low frequencies we want $|S(j\omega)| = |1+L(j\omega)|^{-1}$ to be small.
   - We can write this as $|S(j\omega)| \cdot |W_1(j\omega)| < 1$ 
   - This can be approximated as $|L(j\omega)| > |W_1(j\omega)|$

2. Noise rejection requirement:
   - $|T(j\omega)| \cdot |W_2(j\omega)| < 1$
   - This translates to $|L(j\omega)| < |W_2(j\omega)|^{-1}$

3. Cross-over frequency:
   - The frequency where $|L(j\omega_{gc})| = 1$ is called the cross-over frequency $\omega_{gc}$

Where $W_1(j\omega)$ and $W_2(j\omega)$ are weighting functions (think that they could be another controller) that capture the desired behavior at low and high frequencies, respectively.

These create an "obstacle course" on the Bode plot through which $L(j\omega)$ must navigate to ensure that the frequency specifications are met. The goal would be to find a loop transfer function that stays between the two curves. 

<div style="text-align:center;">
<img src="./img/obstacle.png" alt="Obstacle" width="500">
</div>

The challenge is to design $L(j\omega)$ such that:

   - It stays above $W1$​ at low frequencies
   - It stays below $W_2^{-1}$​ at high frequencies
   - It transitions smoothly between these bounds near crossover

# 2. Loop Shaping



We've seen how different control specifications create a sort of "obstacle course" in the frequency domain. This can feel quite abstract at first - how do we actually shape our loop transfer function L(s) to navigate through these obstacles? 

Let's explore this interactively! Below is a tool that lets you experiment with the fundamental building blocks of loop shaping:

1. **Proportional Gain (k)**:
   - The simplest tool in our toolkit 
   - Shifts the magnitude plot up/down uniformly
   - No effect on phase
   - What happens to stability as we increase gain?

2. **Lead Compensation** ($\frac{s/a + 1}{s/b + 1}, 0 < a < b$):
   - Our tool for boosting phase margin
   - Maximum phase boost occurs around frequency $\sqrt{ab}$
   - Try placing this near your desired crossover frequency!

3. **Lag Compensation** ($\frac{s/a + 1}{s/b + 1}, 0 < b < a$):
   - Great for improving low-frequency performance
   - Needs to be placed well below crossover to avoid phase loss
   - Can you use this to meet the tracking specifications?

The plot shows:
- Plant response in green (try changing it with the numerator/denominator fields!)
- Compensated system in blue 
- Red region: Must stay above this at low frequencies for good tracking
- Yellow region: Must stay below this at high frequencies to reject noise

Play around with different combinations - what works? What doesn't? Can you find a simple solution that meets all specifications? There's often more than one way to succeed!

In [ ]:
def create_plant(num_str, den_str):
    """Create plant transfer function from string inputs"""
    try:
        num = [float(n) for n in num_str.split(',')]
        den = [float(d) for d in den_str.split(',')]
        return control.TransferFunction(num, den)
    except:
        # Return default plant if there's an error
        return control.TransferFunction([0.01, 0.2, 2], [1, 2, 2, 0])

def create_compensator(K, lead_zero, lead_pole, lag_zero, lag_pole, use_lead=True, use_lag=True):
    """Create compensator transfer function based on parameters"""
    comp = control.TransferFunction([K], [1])  # Start with gain
    
    if use_lead:
        # Lead compensator: (s/a + 1)/(s/b + 1) where b > a
        lead_num = [1/lead_zero, 1]
        lead_den = [1/lead_pole, 1]
        comp = comp * control.TransferFunction(lead_num, lead_den)
    
    if use_lag:
        # Lag compensator: (s/a + 1)/(s/b + 1) where a > b
        lag_num = [1/lag_zero, 1]
        lag_den = [1/lag_pole, 1]
        comp = comp * control.TransferFunction(lag_num, lag_den)
    
    return comp

def plot_bode(K=1, lead_zero=0.3, lead_pole=30, lag_zero=10, lag_pole=0.1, 
              use_lead=True, use_lag=True, show_bounds=True,
              num_str='0.01,0.2,2', den_str='1,2,2,0'):
    """Plot Bode diagram with compensator effects"""
    plt.figure(figsize=(12, 8))
    
    # Create plant and compensator
    plant = create_plant(num_str, den_str)
    comp = create_compensator(K, lead_zero, lead_pole, lag_zero, lag_pole, use_lead, use_lag)
    
    # Get loop transfer function
    L = plant * comp
    
    # Generate frequency points
    w = np.logspace(-2, 3, 1000)
    
    # Get frequency response
    mag, phase, w = control.bode(L, w, plot=False)
    
    # Plot magnitude
    plt.subplot(211)

    # Get and plot plant frequency response
    mag_p, phase_p, _ = control.bode(plant, w, plot=False)
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag_p), 'g-', label='Plant', alpha=0.7)

    plt.subplot(212)
    plt.semilogx(w, phase_p * 180/np.pi, 'g-', alpha=0.7)

    # Then plot L(s) in blue
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag), 'b-', label='Compensated system')
    
    # Add performance bounds
    if show_bounds:
        # W1 bound (low-frequency box for disturbance rejection)
        # Must stay ABOVE this at low frequencies
        w1_x = [w[0], 0.1, 0.1, w[0]]
        w1_y = [20, 20, -100, -100]
        plt.fill(w1_x, w1_y, alpha=0.5, color='r', label='Load disturbance\nattenuation')

        # W2 bound (high-frequency box for noise rejection)
        # Must stay BELOW this at high frequencies
        w2_x = [100, w[-1], w[-1], 100]
        w2_y = [100, 100, -25, -25]
        plt.fill(w2_x, w2_y, alpha=0.5, color='y', label='High frequency\nmeasurement noise')

    plt.grid(True)
    plt.ylabel('Magnitude [dB]')
    plt.legend()
    
    # Add reference lines
    plt.axhline(y=0, color='k', linestyle=':')
    
    # Plot phase
    plt.subplot(212)
    plt.semilogx(w, phase * 180/np.pi, 'b-')
    plt.grid(True)
    plt.ylabel('Phase [deg]')
    plt.xlabel('Frequency [rad/s]')
    
    # Add phase margin indicators
    gain_crossover_idx = np.abs(mag - 1).argmin()
    if np.abs(mag[gain_crossover_idx] - 1) < 0.1:  # Only show if we actually cross 0dB
        gain_crossover = w[gain_crossover_idx]
        phase_at_crossover = phase[gain_crossover_idx] * 180/np.pi
        phase_margin = 180 + phase_at_crossover
        
        plt.subplot(211)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        
        plt.subplot(212)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        plt.plot(gain_crossover, phase_at_crossover, 'ro')
        plt.text(gain_crossover, phase_at_crossover, f'PM = {phase_margin:.1f}°')
        
        # Add reference lines
        plt.axhline(y=-180, color='k', linestyle=':')
    
    plt.tight_layout()

# Create interactive widgets
numerator_input = widgets.Text(
    value='0.01,0.2,2',
    description='Numerator:',
    style={'description_width': 'initial'}
)

denominator_input = widgets.Text(
    value='1,2,2,0',
    description='Denominator:',
    style={'description_width': 'initial'}
)

K_slider = FloatSlider(value=1, min=0.1, max=1000, step=0.1, description='Gain K')
lead_zero_slider = FloatSlider(value=0.3, min=0.01, max=10, step=0.1, description='Lead Zero')
lead_pole_slider = FloatSlider(value=30, min=1, max=100, step=1, description='Lead Pole')
lag_zero_slider = FloatSlider(value=10, min=0.1, max=50, step=0.1, description='Lag Zero')
lag_pole_slider = FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='Lag Pole')
use_lead = widgets.Checkbox(value=True, description='Use Lead')
use_lag = widgets.Checkbox(value=True, description='Use Lag')
show_bounds = widgets.Checkbox(value=True, description='Show Bounds')

# Create interactive plot
interactive_plot = interactive_output(plot_bode, 
                                   {'K': K_slider,
                                    'lead_zero': lead_zero_slider,
                                    'lead_pole': lead_pole_slider,
                                    'lag_zero': lag_zero_slider,
                                    'lag_pole': lag_pole_slider,
                                    'use_lead': use_lead,
                                    'use_lag': use_lag,
                                    'show_bounds': show_bounds,
                                    'num_str': numerator_input,
                                    'den_str': denominator_input})

# Display widgets and plot
display(VBox([
    HBox([numerator_input, denominator_input]),
    HBox([K_slider]),
    HBox([use_lead, lead_zero_slider, lead_pole_slider]),
    HBox([use_lag, lag_zero_slider, lag_pole_slider]),
    HBox([show_bounds]),
    interactive_plot
]))

## 2.2 Limitations: Unstable and non-minimum-phase systems



You might have noticed in our exploration above that achieving all specifications simultaneously can be challenging. In fact, this hints at some fundamental limitations in control design that become even more pronounced for certain types of systems.

Consider for example a system with a non-minimum phase zero (a zero in the right half-plane). We can always factor such a system as P(s) = Pmp(s) · D(s), where Pmp(s) is a nicely behaved minimum-phase system and D(s) is what we might call a "Bode demon" - an all-pass filter that doesn't affect the magnitude but introduces an unavoidable phase lag. Specifically, it adds a phase lag of -2arctan(ω/z), reducing our achievable phase margin. The practical consequence? We're forced to cross 0dB at lower frequencies, effectively slowing down our closed-loop system. The slower the zero (small z), the worse this limitation becomes.

A similar story unfolds for unstable open-loop systems (those with poles in the right half-plane), but with a twist. Here the "Bode demon" adds a phase lag of -2arctan(p/ω). To stabilize such a system, we must cross 0dB at higher frequencies, requiring high bandwidth and gain. This translates to needing fast controllers and powerful actuators in practice. And just as slow non-minimum phase zeros are troublesome, fast unstable poles (large p) pose greater challenges.

These aren't just practical difficulties - they represent fundamental limitations in control design, beautifully captured in Bode's integral theorem:

$$\int_0^{\infty} \log|S(j\omega)| d\omega = \pi\sum p_k$$

This elegant formula tells us something profound: if we want to reject disturbances in one frequency range (small sensitivity S), we must necessarily amplify them somewhere else. It's a mathematical expression of the "waterbed effect" - push down one part of the sensitivity function, and another part must rise. The presence of unstable poles makes this tradeoff even more severe.

This is what makes control design both challenging and fascinating. There's rarely a "perfect" solution - instead, we must carefully balance tradeoffs based on what matters most for our specific application. The interactive tool above lets you explore these tradeoffs in a simplified setting, but keeping these fundamental limitations in mind helps us set realistic expectations and make informed design choices in practice.

Let's visualize this waterbed effect directly. In the plot below, we have a simple feedback system with a PI controller. Try increasing the controller gain - as you make the sensitivity smaller at low frequencies (better disturbance rejection), watch how it inevitably increases at higher frequencies (worse noise rejection). This is the waterbed effect in action, and it's a fundamental limitation that no clever control design can circumvent!

Notice how the total area between the sensitivity curve and the 0dB line remains roughly constant - this is exactly what Bode's integral theorem tells us. When we have unstable poles, this area must become even larger, making our design tradeoffs even more challenging.

In [ ]:
def plot_waterbed_demo(gain=1):
    """
    Demonstration of the waterbed effect using real transfer functions
    """
    # Generate frequency points (linear scale)
    w = np.linspace(0.01, 100, 1000)

    # Create a simple plant
    # P(s) = 1/(s(s+1))
    plant = control.TransferFunction([1], [1, 1, 0])

    # Create controller with varying gain
    # C(s) = K - P controller
    controller = control.TransferFunction([gain], [1])

    # Calculate loop transfer function
    L = plant * controller

    # Calculate sensitivity
    S = control.feedback(1, L)

    # Get frequency response
    mag_S, phase_S, _ = control.bode(S, w, plot=False)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))

    # Sensitivity plot
    ax.plot(w, 20*np.log10(mag_S), 'b-', label='Sensitivity S')

    # Add 0dB line
    ax.axhline(y=0, color='k', linestyle=':')

    # Add shading to show area
    ax.fill_between(w, 0, 20*np.log10(mag_S), alpha=0.2, color='blue')

    # Calculate integral correctly
    # Using natural log for the integral
    log_S = np.log(abs(mag_S))  # natural log


    # The integral should be over all frequencies, but we approximate with our finite range
    integral = np.trapezoid(log_S, w)  # integrate
    integral = integral / np.pi  # normalize by π

    ax.set_title(f'Waterbed Effect (∫log|S(jω)|dlog(ω)/π ≈ {integral:.2f})')
    ax.grid(True)
    ax.set_ylabel('Magnitude [dB]')
    ax.set_xlabel('Frequency [rad/s]')
    ax.set_ylim([-40, 40])
    plt.tight_layout()

# Create slider for controller gain
gain_slider = FloatSlider(
    value=1,
    min=0.1, max=1000, step=0.1,
    description='Controller Gain'
)

# Create interactive plot
interactive_plot = interactive_output(plot_waterbed_demo,
    {'gain': gain_slider})

# Display widgets and plot
display(VBox([
    gain_slider,
    interactive_plot
]))